In [1]:
pip install openai --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from openai import OpenAI
import time

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-proj-------------------------------------------------"))

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(assistant_id, user_input):
    thread = client.beta.threads.create()
    run = submit_message(assistant_id, thread, user_input)
    return thread, run

# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

# Pretty printing helper
def pretty_print(messages):
    print("# Assistant Messages")
    for m in messages:
        if m.role == "assistant":
            print(f"{m.role}: {m.content[0].text.value}")
    print()

In [3]:
import json
import pandas as pd

# Function to extract the assistant's JSON message
def extract_json_from_messages(messages):
    for message in messages:
        if message.role == "assistant":
            for content in message.content:
                if '```json' in content.text.value:
                    json_str = content.text.value.split('```json')[1].split('```')[0].strip()
                    return json_str
    return None

# Function to parse the JSON response and organize the data, including case_id
def parse_and_save_response(api_response_json, case_id, thread_id, excel_filename='medical_tests.xlsx'):
    # Parse the JSON response
    response_data = json.loads(api_response_json)

    # Check if response_data is a list
    if not isinstance(response_data, list):
        raise ValueError("Parsed JSON is not a list")

    # Extracting test details
    data = []
    for test in response_data:
        data.append({
            "case_id": test.get('case_id', case_id),  # Use case_id from test or provided case_id
            "thread_id": thread_id,  # Add thread_id to distinguish between runs
            "test_type": test['test_type'],
            "test_name": test['test_name'],
            "test_code": test.get('test_code', ''),
            "order_category": test.get('order_category', ''),
            "clinical_reasoning": test.get('clinical_reasoning', ''),
            "evidence_based_rule": test.get('evidence_based_rule', ''),
            "references": ', '.join(test.get('references', []))
        })

    # Creating a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)

    # Append to existing Excel file or create a new one if it doesn't exist
    try:
        existing_df = pd.read_excel(excel_filename)
        df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        df.to_excel(excel_filename, index=False)
    else:
        df.to_excel(excel_filename, index=False)

    print(f"{test.get('case_id', case_id)}: {thread_id}, Data saved to {excel_filename}")

In [13]:
def recommend_tests(assistant_id, case_id):
    thread, run = create_thread_and_run(assistant_id, case_id)
    thread_id = thread.id
    run = wait_on_run(run, thread)
    #pretty_print(get_response(thread))
    run2 = submit_message(assistant_id, thread, "please provide the recommendation details in json")
    wait_on_run(run2, thread)
    # Fetch the response directly
    api_response = get_response(thread)
    # Extract the last assistant message containing JSON
    last_assistant_message = None
    for message in api_response:
        if message.role == "assistant":
            last_assistant_message = message.content[0].text.value  # Assuming text-based message content
    # Call the other assistant to correct the test codes
    thread, run = create_thread_and_run("asst_5Ldfiv8txpAOgLUyq93XmqCB", last_assistant_message)
    run = wait_on_run(run, thread)
    api_response_v2 = get_response(thread)
    # Extract JSON string from the response messages
    api_response_json = extract_json_from_messages(api_response_v2.data)
    # Define the filename based on assistant_id
    excel_filename = f'recommended_tests_{assistant_id}.xlsx'
    # Parse the response and save to Excel if JSON is found
    if api_response_json:
        try:
            parse_and_save_response(api_response_json, case_id, thread_id, excel_filename)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {str(e)}")
    else:
        print("No valid JSON content found in the assistant's messages.")

In [14]:
# Import case definitions from the cases package
from cases.all_cases import case_gp_s1, case_gp_s2, case_gp_s3, case_gp_s4, case_gp_s5
from cases.all_cases import case_gp_s6, case_gp_s7, case_gp_s8, case_gp_s9, case_gp_s10 
from cases.all_cases import case_er_s1, case_er_s2, case_er_s3, case_er_s4, case_er_s5

# Prompting Strategies
Different OpenAI assistants were created through OpenAI platform https://platform.openai.com/assistants

### Assistant_1 = "Zero Shot Recommender"
#### Instructions=
This assistant serves as a tool for healthcare professionals to navigate clinical decisions with a structured and evidence-based approach for recommending laboratory tests, enhancing patient care quality.

Input: Clinical case scenarios at the gatekeepers of the healthcare system (General Practice and Emergency Room) within specified clinical specialties or multi-specialty contexts. Each case ID starts with "GP" for GP visits or "ER" for ER admissions.

Output: 
1. List of recommended essential tests (with just the names), which are critical for diagnosis or management. They must fit with the following criteria:
   -	Strong evidence from clinical guidelines, systematic reviews, or consensus statements
   -	Crucial information that significantly influences clinical decision-making
   -	Necessary for monitoring the condition or treatment to prevent adverse outcomes or to ensure patient safety.
   - 	Cost-effective, considering its impact on diagnosis, treatment decisions, and outcomes
2. List of recommended conditional tests (with just the names), which may provide useful information but are not critical. They have the following criteria:
   -	Variability in evidence or expert opinion regarding the test's utility, with recommendations possibly depending on patient- specific factors (e.g., age, gender, comorbidities), severity of the condition, or specific clinical context.
   -	Considered a secondary option when primary essential tests are inconclusive, not available, or contraindicated
   -	potential benefits in the given context should be weighed against its costs, risks, and the likelihood of altering clinical management
3.	Only when the user asks about the recommendation details, include the following details for each recommended test/test Panel:
   - LOINC codes and common names
   - Order Category: Classified as Screening, Monitoring, or Diagnosis.
   - Clinical Reasoning: Explanation for ordering this specific test in the given case.
   - Evidence-based Rule: A general rule for recommending this specific test, citing guidelines from reputable medical associations.
   - References: Listed in Vancouver format, pointing to relevant clinical guidelines or publications.
4. Only when the user asks about the recommendation details, provide these additional Components if relevant:
   - Decision Support Tools: If possible, provide structured decision trees or flowcharts to visually guide test ordering based on the clinical scenario.
   - Test Implications Discussion: Address potential implications of the recommended tests, including costs, availability, and patient-specific factors.
   - Ethical Considerations: Include a section discussing ethical considerations in test ordering, focusing on patient welfare, resource allocation, and informed consent.
   
Considerations:
1.	Don’t recommend test panels unless all constituent tests are deemed necessary
2.	Avoid recommending any unnecessary tests, which has any of the following criteria:
   -	lack of evidence supporting the test's use in the specific clinical context, or evidence suggests the test is not effective
   -	The test provides information that is already available from other sources or tests, adding no additional value to patient care.
   -	The potential risks or harms associated with the test (e.g., radiation exposure, false positives/ negatives) outweigh any potential benefits.
   -	not cost-effective, with minimal or no impact on improving patient outcomes or decision-making.


#### Model= "gpt-4o"
MODEL CONFIGURATION:
- Temperature = 0.1
- Top P = 0.1

In [18]:
zero_shot_recommender_id = "asst_XevRWKsCfz07yPbhi8VtUxz9"

In [36]:
cases = [case_gp_s1, case_gp_s2, case_gp_s3, case_gp_s4, case_gp_s5]

# Loop to run recommend_tests 10 times for each case
for case in cases:
    for i in range(10):  # Run each case 10 times
        recommend_tests(zero_shot_recommender_id, case)

GP-S1: thread_waPRyh9zrRHZgtGsx1g7GOgf, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_FMGWY9VGlUlR1dZGmEmU6F28, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_IgAUZ9VvvZxunhXyCpI8Wr7p, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_RmwRaFhvFnXcNd4NUNOZX3HK, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_fXWdB409tCukNO5FqTGmg1qN, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_UUglxkMV15LNHVoOUlsCFlwW, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_0WWi1WzgUyGN2d8LTyeADkpO, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_BdrN2GGzu3OARC8hVyLs5XiK, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S1: thread_wOdKiYUv3r8PnNlyS6c7yLGE, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
Error decoding JSON: Expecting proper

In [39]:
cases = [case_gp_s6, case_gp_s7, case_gp_s8, case_gp_s9, case_gp_s10]

# Loop to run recommend_tests 10 times for each case
for case in cases:
    for i in range(10):  # Run each case 10 times
        recommend_tests(zero_shot_recommender_id, case)

GP-S6: thread_hBwGX6LwbbUwmha3oh1DdDgx, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_gLNSLxjpijianZBaTE8h8fLX, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_OQL0uGp64a7rsGictQMGJVtn, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_kqFbTIzDvMNh77n9MJBr4AOl, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_UhvLXG5aNY77oDssj2HHo5Y7, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_QHRCWDo7Gey895KhsPraCfHn, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_uMVa0JKd9TOoimbc4GXnLff1, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_erHRxR3JIYQaIGOJ5TPKwjc6, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_IUqEpOp3Hpyy4vk3iI8uMvcF, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
GP-S6: thread_Ofh2SdMDDL2a8heGlbtccjQ

In [ ]:
cases = [case_er_s1, case_er_s2, case_er_s3, case_er_s4, case_er_s5]

# Loop to run recommend_tests 10 times for each case
for case in cases:
    for i in range(10):  # Run each case 10 times
        recommend_tests(zero_shot_recommender_id, case)

## Assistant_2 = "CoT Reasoning Recommender"
#### Instructions=
This assistant serves as a tool for healthcare professionals to navigate clinical decisions with a structured and evidence-based approach for recommending laboratory tests, enhancing patient care quality.

Input: Clinical case scenarios at the gatekeepers of the healthcare system (General Practice and Emergency Room) within specified clinical specialties or multi-specialty contexts. Each case ID starts with "GP" for GP visits or "ER" for ER admissions.

Use the following step-by-step instructions to recommend laboratory tests. Provide only the output requested not all the step details.

Step 1 - Understand the Clinical Context:
Analyze the provided details to understand the clinical scenario. 

Step 2 - Consider these three categories for ordering a laboratory test:
1. Diagnosis:
   - Use step-by-step hypothesis deduction to establish a differential diagnosis based on the key clinical findings of the given case. Compile a list of the suspected diagnoses with a prefix that says "Differential Diagnosis: ". Order them to start with the most likely one.
   - Select the most appropriate laboratory tests to confirm or rule out the suspected diagnoses. 
   - Example: 
     - User: a child presenting with fever and seizures. 
     - Assistant: 
       Differential Diagnosis: meningitis, febrile seizures. 
       Recommended lab tests for Diagnosis: CSF WBC/RBC counts, CSF total protein, CSF glucose, CSF culture, and CSF PCR for meningococcus.

2. Monitoring (eg, the effect of drug therapy).
   - Example: 
     - User: a 45-year-old female with Type 2 Diabetes Mellitus (diagnosed 15 years ago, currently on metformin) with no recent lab tests for 6 months
     - Assistant: 
       Recommended lab tests for monitoring: HbA1c, Lipid Profile 
  
3. Screening (eg, for congenital hypothyroidism via neonatal thyroxine testing).
   - Example: 
     - User: a 55-year-old male with no previous PSA test
     - Assistant: 
       Recommended lab tests for screening: prostate-specific antigen 

Step 3 - Provide a structured decision tree or algorithm to order recommended lab tests for establishing a diagnosis. Use logical and sequential decision-making to minimize unnecessary tests, and increase efficiency.

Step 4 - From the initially recommended lab tests, filter out any unnecessary tests using Rifle vs. Shotgun Approach to target specific tests with high diagnostic accuracy and avoid ordering a broad array of tests indiscriminately. In that sense, don’t recommend test panels unless all constituent tests are deemed necessary. Provide the list of filtered-out unnecessary tests, if any.
   - Example: For a child with URI and seizures, order targeted CSF tests rather than a broad panel of unrelated tests (Quantitative immunoglobulins (IgG, IgA, IgM), Erythrocyte sedimentation rate (ESR), Quantitative alpha-1-antitrypsin (AAT) level, Retic count, Arterial blood gasses (ABGs), Throat culture, Sweat chloride, Nasal smear for eosinophils, Nasopharyngeal culture for pertussis infection, Viral cultures, Stool exam for ova and parasites (O & P), Urinalysis, Purified protein derivative (tuberculin) (PPD)/ trichophyton/cocci skin tests, Electrolytes, Glucose, Total bilirubin, Aspartate aminotransferase (AST), Alanine aminotransferase (ALT))

Step 5 - Provide a list of recommended essential tests (with just the names), which are critical for diagnosis or management. They must fit with the following criteria:
 - Strong evidence from clinical guidelines, systematic reviews, or consensus statements
 - Crucial information that significantly influences clinical decision-making
 - Necessary for monitoring the condition or treatment to prevent adverse outcomes or to ensure patient safety.
 - Cost-effective, considering its impact on diagnosis, treatment decisions, and outcomes

Step 6 - Provide a list of recommended conditional tests (with just the names), which may provide useful information but are not critical. They have the following criteria:
 - Variability in evidence or expert opinion regarding the test's utility, with recommendations possibly depending on patient-specific factors (e.g., age, gender, comorbidities), severity of the condition, or specific clinical context.
 - Considered a secondary option when primary essential tests are inconclusive, not available, or contraindicated
 - potential benefits in the given context should be weighed against its costs, risks, and the likelihood of altering clinical management

Step 7 - Only when the user asks about the recommendation details, include the following details for each recommended test/test Panel:
 - LOINC codes and common names
 - Order Category: Classified as Screening, Monitoring, or Diagnosis.
 - Clinical Reasoning: Explanation for ordering this specific test in the given case.
 - Evidence-based Rule: A general rule for recommending this specific test, citing guidelines from reputable medical associations.
 - References: Listed in Vancouver format, pointing to relevant clinical guidelines or publications.

#### Model= "gpt-4o"
MODEL CONFIGURATION:
- Temperature = 0.1
- Top P = 0.1

In [25]:
cot_recommender_id = "asst_FsskhpyafxhnE3rsFhAEFbI8"

In [41]:
cases = [case_gp_s1, case_gp_s2, case_gp_s3, case_gp_s4, case_gp_s5]

# Loop to run recommend_tests 10 times for each case
for case in cases:
    for i in range(10):  # Run each case 10 times
        recommend_tests(cot_recommender_id, case)

GP-S1: thread_9SqvjHVzlRaUyOgmeBlwADfU, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_eh5p3AZI8C1Ut8hHg5kBgc9k, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_ylVIqreOynmCefkuS6Ya45nA, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_iXNeoSflcfh5V5R1AcJavREF, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_FQQ3KKNBi9IVHZd1MLICw9mV, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_2hRvSdQ7YDSiooueBufp0QAF, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_IcvIU1YoCZplIb0YlcfdYeo0, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_GHymEpaMWt0YIO6pbK7VOHdF, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_spIqvVMMfNGT4XgRHbUoHoeQ, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S1: thread_Q3T7T7fPmpoZZ4eudxaotGB

ValueError: Parsed JSON is not a list

In [44]:
cases = [case_gp_s6, case_gp_s7, case_gp_s8, case_gp_s9, case_gp_s10]

# Loop to run recommend_tests 10 times for each case
for case in cases:
    for i in range(10):  # Run each case 10 times
        recommend_tests(cot_recommender_id, case)

GP-S6: thread_XYvFjfp286JCQb7NvKsdGhzo, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_umIbWtAPkbUmVZtNVIsE85CG, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_RdsmNaNWSSvUokvP0KgX9HmS, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_VvlzgzxTtIbBrzsyzwNkLkkd, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_k6obYU2od4W4jDnqS3fTQdgK, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_d4IEeIBZZZXNfFwmRhIfsDqy, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_ais7MvT4PViMtN5VVX4rVgg9, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_u71Xl8WJggrm0d2AoUWNrBH4, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_n9liTNMvLOX8XGEqB17Mko3L, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
GP-S6: thread_7cnffSectJaQgaTm7XxORxC

IndexError: list index out of range

In [46]:
cases = [case_er_s1, case_er_s2, case_er_s3, case_er_s4, case_er_s5]

# Loop to run recommend_tests 10 times for each case
for case in cases:
    for i in range(10):  # Run each case 10 times
        recommend_tests(cot_recommender_id, case)

ER-S1: thread_ywBPeaPLxSeJ5MDkCLtqVo5Q, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
ER-S1: thread_Plln7UbLPN75oaVZDi1haSvr, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
ER-S1: thread_3nTsnDI4JOVKYlD5njYz1OJz, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
ER-S1: thread_YTDjEhW4kCbOgFqgzzyssVQf, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
ER-S1: thread_Br8336IdFz4rS8C0rk4zVQUT, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
ER-S1: thread_U7uznyeq5Cxg3dBPKRhIDYF5, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
No valid JSON content found in the assistant's messages.
ER-S1: thread_8f9r9XBGRgQMj91fcuCnVaW2, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
ER-S1: thread_JUTZAtBaSRiC5ujDP4ntMeYX, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx
ER-S1: thread_Lmu3dKtswJuz0zFvyaO62uCe, Data saved to recommended_tests_asst_Fsskhpyafx

KeyError: 'test_name'

In [50]:
cases = [case_er_s1, case_er_s2, case_er_s3, case_er_s4, case_er_s5]

# Loop to run recommend_tests 10 times for each case
for case in cases:
    for i in range(10):  # Run each case 10 times
        recommend_tests(zero_shot_recommender_id, case)

ER-S1: thread_t2DvofZUj6qY3ylpj3BXv0vs, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_QdmP78Se5jRngHxfqz68VkGs, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_LypTV0gekGl3AOMVdekOI5cr, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_64V2vqQKsNzRwtr8gTCvdSIU, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_2q7nbUR6sVq1Rf62LFybEPES, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_XccPRtQYccBCjF16tPVfB0yz, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_sYQEV3005s23TNC6AZYpcaQ8, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_y0XqdIuKxf76NehISdiDLeTQ, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_KtPfbtS1FaPVBrf4Opa0IM4b, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S1: thread_Bfw01w6Xd2QsR9CAlEepNk4

KeyError: 'test_name'

In [51]:
recommend_tests(cot_recommender_id, case_er_s1)

ER-S1: thread_uj2MS8QqXzXEaEzKuavzYC6d, Data saved to recommended_tests_asst_FsskhpyafxhnE3rsFhAEFbI8.xlsx


In [52]:
recommend_tests(zero_shot_recommender_id, case_er_s2)
recommend_tests(zero_shot_recommender_id, case_er_s2)
recommend_tests(zero_shot_recommender_id, case_er_s5)

ER-S2: thread_MYZvKZhiG94n25DVEpQkWDNt, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S2: thread_H76Hc7PuoxkgvjfHqkayjSVG, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
ER-S5: thread_O4fIkGyidMXTReMrw4pHKOtq, Data saved to recommended_tests_asst_XevRWKsCfz07yPbhi8VtUxz9.xlsx
